<a href="https://colab.research.google.com/github/hoangcuongnguyen2001/RAG_lessons/blob/main/Evaluating_RAG_with_DeepEval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Đánh giá RAG (hoặc LLMs) với DeepEval**

Nhiều chương trình đã được phát triển để đánh giá RAG systems trong giai đoạn gần đây, như DeepEval, RAGAS và TRULens. Phần này chúng ta sẽ sử dụng DeepEval, một mô hình phát triển bởi start-up Confident AI: https://docs.confident-ai.com/.

Lý do chúng ta sử dụng DeepEval trong topic này như sau:

1. DeepEval là một mô hình mã nguồn mở, nên bất kì ai có thể sử dụng mà không cần mất phí (khác với các platforms của các công ty lớn như Amazon Bedrock).
2. DeepEval hỗ trợ đo đạc nhiều metric khác nhau, như thiên vị (bias), độ độc hại của câu trả lời (toxicity), mức độ liên quan với câu hỏi (answer relevancy).
3. Mọi người có thể sử dụng DeepEval chỉ với một câu hỏi và một câu trả lời từ LLM, thay vì dùng cả một dataset với một số platform khác.


***Cài đặt các thư viện cần thiết:***



Chúng ta trước hết cần cài đặt các thư viện chính, để sử dụng DeepEval.

Lưu ý là vì DeepEval cần một LLM nhất định, chúng ta cần phải cài thêm các thư viện cần thiết cho LLM, như `transformers`, `torch`, `bitsandbytes`, `accelerate`.

In [ ]:
!pip install deepeval

In [ ]:
!pip install transformers torch bitsandbytes accelerate

Với LLMs, formats DeepEval cần sử dụng để đọc input là JSON. Có một số trường hợp do model quá cũ, prompt engineering sẽ không thể tạo ra JSON format phù hợp. Chúng ta có thể gặp lỗi sau đây:

`ValueError: Evaluation LLM outputted an invalid JSON. Please use a better evaluation model.`

Do đó, chúng ta cần một phương pháp để thay đổi kết quả của các LLMs cho phù hợp, bằng thư viện `lm-format-enforcer`.

*Lưu ý:* Với các mô hình của các hãng lớn (GPT-4, Gemini,...) vấn đề này sẽ không xảy ra, nên chúng ta có thể bỏ qua bước này.




In [ ]:
!pip install lm-format-enforcer

Bước tiếp theo của chúng ta là tải LLMs xuống GPU cho DeepEval sử dụng. Chúng ta sẽ tuân theo các bước sau:

1. Thừa kế DeepEvalBaseLLM.
2. Sử dụng `get_model_name()` method. Nó sẽ trả về tên của model chúng ta đang sử dụng (trong ví dụ này, chúng ta sẽ dùng Llama 3).
3. Sử dụng `load_model()` method, nhằm trả về một đối tượng (object) liên quan tới model.
4. Sử dụng generate() method, với 2 input chính: string (prompt), và schema (từ BaseModel). Lưu ý: BaseModel được tạo bởi thư viện pydantic, một thư viện tạo kiểu dữ liệu phổ biến trên Python.
5. Chúng ta gọi parser để thay đổi prompt và kết quả cho phù hợp với yêu cầu của DeepEval, sử dụng JsonSchemaParser từ lm-format-enforcer.
6. Một method khác, a_generate(prompt) cần được viết, giống như generate(prompt), nhưng diễn ra không đồng thời với method gốc (nên được gọi là asynchronous (async) method). Trong ví dụ này, method này chỉ dùng lại generate(prompt), nhưng các bạn có thể thay đổi để cho việc đánh giá LLMs nhanh hơn.

Các bạn có thể tham khảo post này của DeepEval để tìm hiểu thêm: https://docs.confident-ai.com/docs/guides-using-custom-llms

In [ ]:
import transformers
import torch
from pydantic import BaseModel
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import (
    build_transformers_prefix_allowed_tokens_fn,
)
import json
from deepeval.models import DeepEvalBaseLLM


class CustomLlama3_8B(DeepEvalBaseLLM):
    def __init__(self):
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
        )

        model_4bit = AutoModelForCausalLM.from_pretrained(
            "meta-llama/Meta-Llama-3-8B-Instruct",
            device_map="auto",
            quantization_config=quantization_config,
        )
        tokenizer = AutoTokenizer.from_pretrained(
            "meta-llama/Meta-Llama-3-8B-Instruct"
        )

        self.model = model_4bit
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        model = self.load_model()

        pipeline = transformers.pipeline(
            "text-generation",
            model=model,
            tokenizer=self.tokenizer,
            use_cache=True,
            device_map="auto",
            max_length=2500,
            do_sample=True,
            top_k=5,
            num_return_sequences=1,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.eos_token_id,
            truncation=True,
        )

        # Create parser required for JSON confinement using lmformatenforcer
        parser = JsonSchemaParser(schema.schema())
        prefix_function = build_transformers_prefix_allowed_tokens_fn(
            pipeline.tokenizer, parser
        )

        # Output and load valid JSON
        output_dict = pipeline(prompt, prefix_allowed_tokens_fn=prefix_function)
        output = output_dict[0]["generated_text"][len(prompt) :]
        json_result = json.loads(output)

        # Return valid JSON object according to the schema DeepEval supplied
        return schema(**json_result)

    async def a_generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        return self.generate(prompt, schema)

    def get_model_name(self):
        return "Llama-3 8B"

***Một số ví dụ về metric dùng trong đánh giá RAG:***

*Mức độ liên quan với câu hỏi (Answer Relevancy Metric):*

Đây là một metric đo xem câu trả lời `(actual_output)` của RAG có liên quan gì tới câu hỏi `(input)` hay không. Nó là một số thập phân từ 0 tới 1, đo bằng cách sau đây:


> Answer Relevancy Metric = Số lượng câu trả về liên quan tới câu hỏi/Tổng số câu trả về từ RAG.

DeepEval sẽ dùng LLM (trong ví dụ này là Llama3) để tìm tất cả các câu trong `actual_output`, rồi xếp hạng xem nó có liên quan tới `input` hay không.




- **Lưu ý:** Có 6 parameters cần thiết cho metric này, chúng ta có thể thêm hoặc không:

- [Optional] threshold: Một số thập phân cho số nhâ a float representing the minimum passing threshold, defaulted to 0.5.
[Optional] model: a string specifying which of OpenAI's GPT models to use, OR any custom LLM model of type DeepEvalBaseLLM. Defaulted to 'gpt-4o'.
[Optional] include_reason: a boolean which when set to True, will include a reason for its evaluation score. Defaulted to True.
[Optional] strict_mode: a boolean which when set to True, enforces a binary metric score: 1 for perfection, 0 otherwise. It also overrides the current threshold and sets it to 1. Defaulted to False.
[Optional] async_mode: a boolean which when set to True, enables concurrent execution within the measure() method. Defaulted to True.
[Optional] verbose_mode: a boolean which when set to True, prints the intermediate steps used to calculate said metric to the console, as outlined in the How Is It Calculated section. Defaulted to False.

In [ ]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase

# evaluate a single test case
actual_output = r'''The Transformer model architecture is described in the paper "Attention is All You Need" [1]. The model consists of an encoder and a decoder,
both of which are composed of stacked self-attention and point-wise, fully connected layers. The encoder has six identical layers, each with two sub-layers:
a multi-head self-attention mechanism and a simple, position-wise fully connected feed-forward network. The decoder has an additional sub-layer that performs
multi-head attention over the output of the encoder stack. Residual connections and layer normalization are employed around each sub-layer. The attention function
 maps a query and a set of key-value pairs to an output, which is a weighted sum of the values, where the weights are computed based on the similarity between the query and the keys.
The model achieves state-of-the-art results on several natural language processing tasks, including machine translation and constituency parsing.


'''

custom_llm = CustomLlama3_8B()
metric = AnswerRelevancyMetric(
    threshold=0.7,
    model=custom_llm,
    include_reason=True
)
test_case = LLMTestCase(
    input="Could you summarize the model architecture of Transformers based on 'Attention is all you need' paper?",
    actual_output=actual_output
)

metric.measure(test_case)
print(metric.score)
print(metric.reason)



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

1.0
The score is 1.00 because the actual output directly addresses the question and provides a concise summary of the Transformer model architecture, with no irrelevant statements or information that is not related to the topic. The output is highly relevant to the input and accurately answers the question.


In [ ]:
from deepeval.metrics import BiasMetric


metric = BiasMetric(threshold=0.5, model=custom_llm)
test_case = LLMTestCase(
    input="Could you summarize the model architecture of Transformers based on 'Attention is all you need' paper?",
    # Replace this with the actual output from your LLM application
    actual_output=actual_output
)

metric.measure(test_case)
print(metric.score)
print(metric.reason)


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

0



In [ ]:
from deepeval.metrics import HallucinationMetric

# Replace this with the actual documents that you are passing as input to your LLM.
context=[r'''In addition, we are taking actions to optimize our global office space. As a result we expect to incur exit costs
relating to office space reductions of approximately $0.5 billion in the first quarter of 2023. We may incur
additional charges in the future as we further evaluate our real estate needs.
•In January 2023, we completed an assessment of the useful lives of our servers and network equipment,
resulting in a change in the estimated useful life of our servers and certain network equipment to six years,
which we expect to result in a reduction of depreciation of approximately $3.4 billion for the full fiscal year 2023
for assets in service as of December 31, 2022, recorded primarily in cost of revenues and R&D expenses.
•As AI is critical to delivering our mission of bringing our breakthrough innovations into the real world, beginning
in January 2023, we will update our segment reporting relating to certain of Alphabet's AI activities. DeepMind,
previously reported within Other Bets, will be reported as part of Alphabet's corporate costs, reflecting its
increasing collaboration with Google Services, Google Cloud, and Other Bets. Prior periods will be recast to
conform to the revised presentation. See Note 15 of the Notes to Consolidated Financial Statements included
in Item 8 of this Annual Report on Form 10-K for information relating to our segments.

''']

# Replace this with the actual output from your LLM application
actual_output=r'''Alphabet in 2023 had to deal with these challenges: Pausing activities and reduce costs
 (Alphabet needs to reduce the global office space in approximately 0.5 billion USD in 2023);
 assessing the useful lives of its server and network equipment, with an estimated reduction in depreciation expenses
  of approximately $3.4 billion in fiscal 2023. It also have to revalue and reorganize AI operations,
   including DeepMind, to reflect increased collaboration between AI operations and other operations.

'''

test_case = LLMTestCase(
    input="Could you summarize challenges that Alphabet is facing in 2023?",
    actual_output=actual_output,
    context=context
)
metric = HallucinationMetric(threshold=0.5, model=custom_llm, include_reason=True )

metric.measure(test_case)
print(metric.score)
print(metric.reason)

Output()

0.3333333333333333
The score is 0.33 because the actual output lacks detail about the actions taken to optimize the global office space and change in estimated useful life of servers and network equipment, which are mentioned in the context, but the actual output agrees with the context on other points, indicating some hallucination has occurred, but not extensively so, resulting in a relatively low hallucination score of 0.33.
